In [1]:
%cd /content/drive/MyDrive/IIP2/Assignments/Final/HyperParameterOptimization

/content/drive/MyDrive/IIP2/Assignments/Final/HyperParameterOptimization


In [ ]:
# Installing the "Simple Transformer" library
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 30.8 

In [ ]:
# Importing the necessary libraries
import logging
import sklearn
import os
import requests
import json

In [ ]:
# Getting training and testing datasets

os.mkdir('squad')
url = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/'

for file in ['train-v2.0.json', 'dev-v2.0.json']:
  res = requests.get(f'{url}{file}') #make request
  with open(f'squad/{file}', 'wb') as f:
    for chunk in res.iter_content(chunk_size=4):
      f.write(chunk)

In [ ]:
# Preparing a training dataset (130319 samples)

with open("squad/train-v2.0.json", "rb") as f:
  squad_dict = json.load(f)

train = []
for group in squad_dict['data']:
  train = train + group['paragraphs']

# Using a portion of the testing dataset to perform HyperParameter Optimization
train_data1 = train[0:80]

In [ ]:
# Preparing an evaluation dataset (used during the training phase) (5448 samples)
with open("squad/dev-v2.0.json", "rb") as f:
  squad_dict = json.load(f)

eval = []
for i in range(17):
  eval = eval + squad_dict['data'][i]['paragraphs']

# Using a portion of the evaluation dataset to perform HyperParameter Optimization
eval_data1 = eval[0:70]

--------------------------------------
Configuring hyperparameter optimization through W&B Sweeps

In [ ]:
import logging

import pandas as pd
import sklearn

import wandb
from simpletransformers.question_answering import (
    QuestionAnsweringModel,
    QuestionAnsweringArgs,
)

In [ ]:
# Configuring the sweep

sweep_config = {
    # Specify the search strategy
    "method": "bayes",  # grid, random

    # Specify the metric to be optimized
    "metric": {"name": "train_loss", "goal": "minimize"},

    # Specify the hyperparameters and their values to explore
    "parameters": {
        "train_batch_size": {"values": [64, 128] },  # Discret values
        "learning_rate": {"min": 4e-5, "max": 4e-4}, # Range of values
    },
}

In [ ]:
# Initialize a W&B sweep
sweep_id = wandb.sweep(sweep_config, project="HyperParameterFina")

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


In [ ]:
# Set up the default configuration for the model

model_args = QuestionAnsweringArgs()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.manual_seed = 4
model_args.use_multiprocessing = True
# For each exploration 3 epochs are performed
model_args.num_train_epochs = 3
model_args.eval_batch_size = 64
model_args.wandb_project = "HyperParameterFina"

In [ ]:
# Set up the training function

def train1():
    # Initialize a new wandb run
    wandb.init()

    # Create a TransformerModel
    model = QuestionAnsweringModel(
        "roberta",
        "roberta-base",
        use_cuda=True,
        args=model_args,
        sweep_config=wandb.config,
    )

    # Train the model
    model.train_model(train_data=train_data1,eval_data=eval_data1)

    # Evaluate the model
    model.eval_model(eval_data=eval_data1)

    # Sync wandb
    wandb.join()

In [ ]:
# Run the sweeps
wandb.agent(sweep_id, train1)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 69up2rlf
Sweep URL: https://wandb.ai/xhono/HyperParameterFina/sweeps/69up2rlf


wandb: Agent Starting Run: soq18jjo with config:
wandb: 	learning_rate: 0.00025759008553275317
wandb: 	train_batch_size: 128
wandb: Currently logged in as: xhonihoxha53 (xhono). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
add example index and unique id: 100%|██████████| 949/949 [00:00<00:00, 505639.54it/s]


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Running Epoch 0 of 3:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
